In [2]:
import pandas as pd
from sqlite3 import connect

In [3]:
conn = connect("album.db")

In [4]:
df = pd.read_sql('''SELECT a.title AS Album, COUNT(t.track_number) as Tracks
  FROM track AS t
  JOIN album AS a
    ON a.id = t.album_id
  GROUP BY a.id
  ORDER BY Tracks DESC, Album''', conn)

In [5]:
df

,Album,Tracks
0,Rubber Soul,14
1,Birds of Fire,10
2,Two Men with the Blues,10
3,Apostrophe,9
4,Hendrix in the West,9
5,Live And,6
6,Kind of Blue,5
